# Python FMS Demo
This notebook shows an example of how to use FMS to extract magnitude and phases
from audio files. It also compares results to the included reference files, and
for more complete analysis one can run the comparisons in the `tests` folder.

Finally we include some discussion of the differences between MATLAB and Python
reference results.

In [ ]:
import os
import numpy as np
import pickle

from scipy.io import loadmat

import fms

# Demo

In [ ]:
wav_path = "../wavs"
fnames = [
    "audioShort16k.wav",  #     %fs = 16k,      length appx. 1.6 sec
    "audioLong16k.wav",  #      %fs = 16k,      length appx. 3.5 sec
    "audio22k.wav",  #          %fs = 22.05k,   length appx. 3.3 sec
    "audio32k.wav",  #          %fs = 32k,      length appx. 2.9 sec
    "audio44k.wav",  #          %fs = 44.1k,    length appx. 5.8 sec
    "audio48k.wav",  #          %fs = 48k,      length appx. 3.0 sec
]

reference_folder = "reference_files/"
output_folder = "output"

os.makedirs(output_folder, exist_ok=True)

for fname in fnames:
    filepath = os.path.join(wav_path, fname)

    fs, audio = fms.load_audio(filepath)
    # Get FMS representation
    PsiM, PsiP = fms.fms(audio, fs)

    name = os.path.basename(filepath)
    name, _ = os.path.splitext(name)

    outputs = dict(PsiM=PsiM, PsiP=PsiP)
    output_path = os.path.join(output_folder, name + ".pkl")
    with open(output_path, "wb") as output:
        pickle.dump(dict(magnitude=PsiM, phase=PsiP), output)
    # Save csvs
    csvout = output_path.replace(".pkl", "PsiM.csv")
    np.savetxt(csvout, PsiM, delimiter=",")

    csvout = output_path.replace(".pkl", "PsiP.csv")
    np.savetxt(csvout, PsiP, delimiter=",")

ref_name = "audio48kRef.pkl"
ref_path = os.path.join(reference_folder, ref_name)
with open(ref_path, "rb") as f:
    ref = pickle.load(f)

new_name = "audio48k.pkl"
new_path = os.path.join(output_folder, new_name)
with open(new_path, "rb") as f:
    new = pickle.load(f)

for key in ["magnitude", "phase"]:
    abs_error = np.abs(ref[key] - new[key])
    mean_abs_error = np.mean(abs_error)
    print(f"Mean absolute {key} error is: {mean_abs_error} ")

# Differences between MATLAB and Python
It is natural that MATLAB and Python would not generate identical results, however
the difference between the two should be fairly small. 
We include average absolute differences between both for all reference files below.
Magnitudes exhibit very good agreement, while phase has a bit more difference.
This is somewhat expected as phase results can be more sensitive to slight perturbations
due to rounding and precision differences throughout the FMS calculation. 
Overall the results are extremely reasonable.

audio22kRef
* magnitude: 1.9095510762900948e-18
* phase: 0.0005917543414119555

audio48kRef
* magnitude: 2.0700467080180722e-18
* phase: 6.909032736318037e-16

audioShort16kRef
* magnitude: 1.9980279990492865e-18
* phase: 3.381725387529536e-16

audio44kRef
* magnitude: 9.57628061600372e-19
* phase: 4.956249384038512e-16

audioLong16kRef
* magnitude: 3.9059985888628e-18
* phase: 0.00025869504723263515

audio32kRef
* magnitude: 1.4880463059126736e-18
* phase: 5.163154314393145e-16

In [ ]:
mat_refpath = "../matlab/reference_files/"
py_refpath = "reference_files"

mat_refs = os.listdir(mat_refpath)
py_refs = os.listdir(py_refpath)

for refname in py_refs:
    mat_ref = refname.replace('.pkl', '.mat')
    mat_path = os.path.join(mat_refpath, mat_ref)
    mat = loadmat(mat_path)

    pypath = os.path.join(py_refpath, refname)
    with open(pypath, "rb") as f:
        py = pickle.load(f)
    
    mag_diff = np.mean(np.abs(mat['PsiM'] - py['magnitude']))
    phase_diff = np.mean(np.abs(mat['PsiP'] - py['phase']))
    print(f'\n{refname.replace(".pkl", "")}\n* magnitude: {mag_diff}\n* phase: {phase_diff}')